In [9]:
# =====================================================
# 🧹 Data Quality Check – Dashboard de Performance Logística
# Autor: Krisley Gregory
# Descrição: Avalia a qualidade dos dados simulados utilizados
# no painel de Performance Logística (Power BI)
# =====================================================

import pandas as pd
import numpy as np

# -------------------------
# 1. Carregamento das bases
# -------------------------
fato = pd.read_csv(r'C:\Users\kriss\Documents\2. Estudos\Estudos\Análise de Dados\data\data_analytics\fato_operacao_logistica.csv')
dim_trem = pd.read_csv(r'C:\Users\kriss\Documents\2. Estudos\Estudos\Análise de Dados\data\data_analytics\dim_trem.csv')
dim_sb = pd.read_csv(r'C:\Users\kriss\Documents\2. Estudos\Estudos\Análise de Dados\data\data_analytics\dim_sb.csv')

print("✅ Bases carregadas com sucesso!")
print(f"Fato: {fato.shape[0]} registros | Dim_SB: {dim_sb.shape[0]} | Dim_Trem: {dim_trem.shape[0]}\n")

# -------------------------
# 2. Conversão de tipos
# -------------------------
# Garante que colunas numéricas sejam realmente numéricas
for col in ['saturacao', 'tempo_transit', 'tempo_parado']:
    if col in fato.columns:
        fato[col] = pd.to_numeric(fato[col], errors='coerce')

# -------------------------
# 3. Completude
# -------------------------
completude = fato.isnull().mean().round(4) * 100
print("📊 Completude (% de valores nulos por coluna):")
display(completude)

# -------------------------
# 4. Consistência de faixas e tipos
# -------------------------
erros_saturacao = fato.loc[fato['saturacao'] > 150]
erros_tempo = fato.loc[(fato['tempo_transit'] <= 0) | (fato['tempo_parado'] < 0)]

print(f"\n⚠️ Registros com saturação > 150%: {len(erros_saturacao)}")
print(f"⚠️ Registros com tempos inválidos: {len(erros_tempo)}")

# -------------------------
# 5. Validade de datas
# -------------------------
if 'data_operacao' in fato.columns:
    fato['data_operacao'] = pd.to_datetime(fato['data_operacao'], errors='coerce')
    min_data, max_data = fato['data_operacao'].min(), fato['data_operacao'].max()
    if pd.notnull(min_data) and pd.notnull(max_data):
        print(f"\n📅 Período de operação: {min_data.date()} a {max_data.date()}")
    else:
        print("\n📅 Datas inválidas ou ausentes na base de fato.")
else:
    print("\n⚠️ Coluna 'data_operacao' não encontrada na base de fato.")

# -------------------------
# 6. Unicidade
# -------------------------
chaves = ['id_trem', 'id_sb', 'data_operacao']
chaves_presentes = [c for c in chaves if c in fato.columns]
if chaves_presentes:
    duplicados = fato.duplicated(subset=chaves_presentes).sum()
    print(f"\n🔁 Registros duplicados ({' + '.join(chaves_presentes)}): {duplicados}")
else:
    duplicados = np.nan
    print("\n⚠️ Colunas de chave não encontradas para verificação de duplicidade.")

# -------------------------
# 7. Integridade referencial
# -------------------------
sb_invalidos = fato[~fato['id_sb'].isin(dim_sb['id_sb'])] if 'id_sb' in fato.columns and 'id_sb' in dim_sb.columns else pd.DataFrame()
trem_invalidos = fato[~fato['id_trem'].isin(dim_trem['id_trem'])] if 'id_trem' in fato.columns and 'id_trem' in dim_trem.columns else pd.DataFrame()

print(f"\n🔗 Registros com SB inexistente: {len(sb_invalidos)}")
print(f"🔗 Registros com Trem inexistente: {len(trem_invalidos)}")

# -------------------------
# 8. Sumário de Qualidade
# -------------------------
total_registros = len(fato)
metricas = {
    'Completude Média (%)': round(100 - completude.mean(), 2),
    'Saturações Inválidas (%)': round((len(erros_saturacao) / total_registros) * 100, 2),
    'Tempos Inválidos (%)': round((len(erros_tempo) / total_registros) * 100, 2),
    'Duplicados (%)': round((duplicados / total_registros) * 100, 2) if not np.isnan(duplicados) else np.nan,
    'Integridade SB (%)': round(100 - (len(sb_invalidos) / total_registros) * 100, 2),
    'Integridade Trem (%)': round(100 - (len(trem_invalidos) / total_registros) * 100, 2)
}

df_quality = pd.DataFrame(metricas, index=['Qualidade Geral'])
display(df_quality)

# -------------------------
# 9. Conclusão
# -------------------------
print("\n✅ Análise de Data Quality concluída!")
print("Os resultados podem ser adicionados ao README.md para documentar a consistência das bases.")



✅ Bases carregadas com sucesso!
Fato: 300 registros | Dim_SB: 11 | Dim_Trem: 7

📊 Completude (% de valores nulos por coluna):


id_operacao                 0.0
data_operacao               0.0
id_trem                     0.0
id_sb                       0.0
tempo_transit             100.0
tempo_parado              100.0
capacidade_bruta            0.0
carga_transportada          0.0
km_percorrido               0.0
anomalias                   0.0
saturacao                 100.0
eficiencia_operacional      0.0
dtype: float64


⚠️ Registros com saturação > 150%: 0
⚠️ Registros com tempos inválidos: 0

📅 Período de operação: 2024-01-01 a 2024-04-29

🔁 Registros duplicados (id_trem + id_sb + data_operacao): 1

🔗 Registros com SB inexistente: 0
🔗 Registros com Trem inexistente: 0


,Completude Média (%),Saturações Inválidas (%),Tempos Inválidos (%),Duplicados (%),Integridade SB (%),Integridade Trem (%)
Qualidade Geral,75.0,0.0,0.0,0.33,100.0,100.0



✅ Análise de Data Quality concluída!
Os resultados podem ser adicionados ao README.md para documentar a consistência das bases.
